In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

In [ ]:
!pip3 install transformers
!cp /content/drive/MyDrive/fake-news-explainability/utils_fake_news.py .

In [ ]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4
import time
import datetime
import random
import itertools
import os

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import (
    Dataset, 
    DataLoader,
    TensorDataset, 
    random_split, 
    RandomSampler, 
    SequentialSampler)

# Transformers
from transformers import (
    BertModel,
    BertForSequenceClassification,
    BertTokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
    AdamW,
    get_linear_schedule_with_warmup)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

%run utils_fake_news.py

## Model & Training Function

In [ ]:
def train():
    total_t0 = time.time()
    for epoch_i in range(0, epochs):
        
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        t0 = time.time()
        total_train_loss = 0
        bert_model.train()

        for step, batch in enumerate(bert_train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(bert_train_dataloader), elapsed))

            # Unpack batch
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Zero grads
            bert_model.zero_grad()        

            # Forward pass
            output = bert_model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask, 
                                labels=b_labels)
            # Accumulate loss
            total_train_loss += output[0].item()

            # Backward pass
            output[0].backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(bert_model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The bert_optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            bert_optimizer.step()

            # Update the learning rate.
            bert_scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(bert_train_dataloader)            
        
        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))
        
        # Record all statistics from this epoch.
        bert_training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Training Time': training_time,
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## Load Data

In [ ]:
# Run if the data has already been encoded
# Load encoded LIAR dataset
df_train_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/training/liar_train.pt")

In [ ]:
# Load data into dataloader
batch_size = 32

bert_train_dataloader = DataLoader(
            df_train_encode,  # The training samples.
            batch_size = batch_size # Trains with this batch size.
        )

## Training

In [ ]:
# Device
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")

# BERT
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Model
bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           num_labels = 2,
                                                           output_attentions = False,
                                                           output_hidden_states = False
                                                          ).to(device)

# Optimizer
bert_optimizer = AdamW(bert_model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
# Training Params
bert_training_stats = []
epochs = 10
total_steps = len(bert_train_dataloader) * epochs

# Learning rate scheduler.
bert_scheduler = get_linear_schedule_with_warmup(bert_optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
# Train or load pre-trained
bert_model_path = "/content/drive/MyDrive/fake-news-explainability/Models/liar_model2"
    
if os.path.exists(bert_model_path):
    bert_model = BertForSequenceClassification.from_pretrained(
        bert_model_path, num_labels = 2).to(device)
else:
    train()
    bert_model.save_pretrained(bert_model_path)

## Accuracy

In [ ]:
# Load dataset
# df_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/training/liar_test.pt")
df_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/training/liar_valid.pt")

labels = np.array([int(t[2]) for t in df_encode])

with torch.no_grad():
    outputs = bert_model(df_encode.tensors[0].to(device),
                         token_type_ids=None, 
                         attention_mask=df_encode.tensors[1].to(device),
                         labels=df_encode.tensors[2].to(device))
    
# Accuracy
print(f"Accuracy: {flat_accuracy(outputs[1].detach().cpu().numpy(), labels)}")

Accuracy: 0.5864485981308412


## Evaluate

In [ ]:
# Load encoded tensors
# df_pos_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/evaluation/liar_valid_pos.pt")
# df_neg_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/evaluation/liar_valid_neg.pt")

# df_pos_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/evaluation/liar_test_name_orig_filtered.pt")
# df_neg_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/evaluation/liar_test_name_new_filtered.pt")

df_pos_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/evaluation/liar_test_polarity_orig_filtered.pt")
df_neg_encode = torch.load("/content/drive/MyDrive/fake-news-explainability/Data/Encoded/liar/evaluation/liar_test_polarity_new_filtered.pt")

# Generate predictions
with torch.no_grad():
    outputs_pos = bert_model(df_pos_encode.tensors[0].to(device),
                              token_type_ids=None, 
                              attention_mask=df_pos_encode.tensors[1].to(device),
                              labels=df_pos_encode.tensors[2].to(device))
    outputs_neg = bert_model(df_neg_encode.tensors[0].to(device),
                               token_type_ids=None, 
                               attention_mask=df_neg_encode.tensors[1].to(device),
                               labels=df_neg_encode.tensors[2].to(device))

## Metrics

In [ ]:
# Percent Labels Flipped
cf_matrix = confusion_matrix(torch.argmax(outputs_pos[1].cpu(), axis=1), 
                             torch.argmax(outputs_neg[1].cpu(), axis=1))
print(f"Labels Flipped: {cf_matrix[0,1]+cf_matrix[1,0]} of {np.sum(cf_matrix)} ({round(100*(cf_matrix[0,1]+cf_matrix[1,0])/np.sum(cf_matrix),4)}%)")

Labels Flipped: 0 of 14 (0.0%)


In [ ]:
# Average Probability Change
m = nn.Softmax(dim=1)
delta_lst = m(outputs_neg['logits'])[:,1]-m(outputs_pos['logits'])[:,1]
print(f"Average Change: {round(float(torch.mean(delta_lst)),4)} ({round(float(torch.std(delta_lst)),4)})")

Average Change: 0.0267 (0.0644)
